In [1]:
!pip install sqlalchemy

Looking in links: /home/rick446/src/wheelhouse
You should consider upgrading via the '/home/rick446/.virtualenvs/classes/bin/python -m pip install --upgrade pip' command.


In [1]:
import pymysql
import getpass
import pandas as pd
import sqlalchemy as sa

In [3]:
password = getpass.getpass('Admin password?')

Admin password?········


In [2]:
password = 'r*X87M5YhEzzRnE'

In [3]:
host = 'advanced-python.c9zriyyu6mtc.us-east-1.rds.amazonaws.com'
conn = pymysql.connect(
    host=host,
    user='admin',
    port=3306,
    password=password,
)

In [4]:
with conn.cursor() as c:
    c.execute('DROP DATABASE class')
    c.execute("DROP USER 'newuser'")

OperationalError: (1008, "Can't drop database 'class'; database doesn't exist")

In [5]:
with conn.cursor() as c:
    c.execute(f'CREATE DATABASE class')
    c.execute(f"CREATE USER 'newuser' IDENTIFIED BY 'password'")
    c.execute(f"GRANT ALL PRIVILEGES ON class . * TO 'newuser'")

In [6]:
conn = pymysql.connect(
    host=host,
    user='newuser',
    password='password',
    db='class'
)

In [7]:
transactions = pd.read_csv('./data/Sacramentorealestatetransactions.csv')
transactions.head()

,street,city,zip,state,beds,baths,sq__ft,type,sale_date,price,latitude,longitude
0,3526 HIGH ST,SACRAMENTO,95838,CA,2,1,836,Residential,Wed May 21 00:00:00 EDT 2008,59222,38.631913,-121.434879
1,51 OMAHA CT,SACRAMENTO,95823,CA,3,1,1167,Residential,Wed May 21 00:00:00 EDT 2008,68212,38.478902,-121.431028
2,2796 BRANCH ST,SACRAMENTO,95815,CA,2,1,796,Residential,Wed May 21 00:00:00 EDT 2008,68880,38.618305,-121.443839
3,2805 JANETTE WAY,SACRAMENTO,95815,CA,2,1,852,Residential,Wed May 21 00:00:00 EDT 2008,69307,38.616835,-121.439146
4,6001 MCMAHON DR,SACRAMENTO,95824,CA,2,1,797,Residential,Wed May 21 00:00:00 EDT 2008,81900,38.519470,-121.435768


In [8]:
engine = sa.create_engine(f'mysql+pymysql://newuser:password@{host}/class')
transactions.to_sql('transactions', engine)

In [9]:
stocks = pd.read_csv('./data/closing-prices.csv', index_col=0)
stocks.head()

,F,TSLA,GOOG,IBM,AAPL
2014-01-02,12.0890,150.10,NaN,157.6001,72.7741
2014-01-03,12.1438,149.56,NaN,158.5430,71.1756
2014-01-06,12.1986,147.00,NaN,157.9993,71.5637
2014-01-07,12.0420,149.36,NaN,161.1508,71.0516
2014-01-08,12.1673,151.28,NaN,159.6728,71.5019


In [10]:
def segment(symbol):
    stocks_values = pd.DataFrame(stocks[symbol])
    stocks_values.index.name = 'date'
    stocks_values['symbol'] = symbol
    stocks_values = stocks_values.reset_index()
    cols = stocks_values.columns
    print(cols)
    stocks_values.columns = 'date price symbol'.split()
    return stocks_values


In [11]:
stock_table_data = pd.concat([segment(symbol) for symbol in 'F TSLA GOOG IBM AAPL'.split()])
stock_table_data.shape

Index(['date', 'F', 'symbol'], dtype='object')
Index(['date', 'TSLA', 'symbol'], dtype='object')
Index(['date', 'GOOG', 'symbol'], dtype='object')
Index(['date', 'IBM', 'symbol'], dtype='object')
Index(['date', 'AAPL', 'symbol'], dtype='object')


(5035, 3)

In [12]:
stock_table_data.head()

,date,price,symbol
0,2014-01-02,12.0890,F
1,2014-01-03,12.1438,F
2,2014-01-06,12.1986,F
3,2014-01-07,12.0420,F
4,2014-01-08,12.1673,F


In [13]:
import sqlite3
with sqlite3.connect('./data/stocks.db') as scon:
    stock_table_data.to_sql('stocks_data', scon, index=False, if_exists='replace')

In [14]:
states = pd.read_csv('./data/states.csv') # , index_col=['Abbreviation'])
states.head()

,Abbreviation,state,area,pop
0,AL,Alabama,135767,4874747.0
1,AK,Alaska,1723337,739795.0
2,AZ,Arizona,295234,7016270.0
3,AR,Arkansas,137732,3004279.0
4,CA,California,423967,39536653.0


In [15]:
engine = sa.create_engine(f'mysql+pymysql://newuser:password@{host}/class')
states.to_sql('states', engine)